# 1. Packages
Updated to include `imblearn.pipeline.Pipeline` for correct cross-validation with resampling.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline # Use imblearn pipeline to handle resampling

# 2. Dataset

In [ ]:
df = pd.read_csv(r'C:\Users\valen\OneDrive\Escritorio\Juano_VS\Beta-Bank\Data\Churn.csv')
df.columns = df.columns.str.lower()
df = df.drop(['rownumber', 'customerid', 'surname'], axis=1)
print(df.info())

In [ ]:
df[df['tenure']==0].shape

In [ ]:
median = df['tenure'].median()
print(median)

In [ ]:
df['tenure'] = df['tenure'].fillna(median)
df.info()

In [ ]:
print(df.duplicated().sum())

In [ ]:
df_ohe = pd.get_dummies(df, columns=['geography', 'gender'], drop_first=True, dtype=int)
df_ohe.head()

In [ ]:
X = df_ohe.drop('exited', axis=1)
y = df_ohe['exited']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(pd.Series(y_train).value_counts(1))
print(pd.Series(y_test).value_counts(1))

# 3. Model Selection with Pipelines
We use `ImbPipeline` to ensure:
1. **Scaling** happens before resampling (critical for distance-based methods like SMOTE/NearMiss).
2. **Resampling** happens *only* on the training folds during Cross-Validation, preventing data leakage.

In [ ]:
def model_select(estimator, param, features_train, target_train, features_test, target_test):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param,
        cv=cv,
        scoring='roc_auc',
        refit=True,
        n_jobs=-1
    )
    # Note: We pass the original training data. The Pipeline handles scaling/resampling internally for each fold.
    grid_search.fit(features_train, target_train)
    
    print(f'Best Hyperparameters Cross-Validation: {grid_search.best_params_}')
    print(f'Best Score Cross-Validation (ROC AUC): {grid_search.best_score_:.4f}')
    
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(features_test)
    probs = best_model.predict_proba(features_test)[:, 1]
    
    print(f'F1 Score Test: {f1_score(target_test, predictions):.4f}')
    print(f'ROC AUC Score Test: {roc_auc_score(target_test, probs):.4f}')
    return best_model

## Decision Tree

In [ ]:
# Define base parameters (note the 'model__' prefix for pipeline compatibility)
param_grid_tree = {
    'model__max_depth': [3, 5, 7, 10],
    'model__min_samples_leaf': [20, 50, 100],
    'model__criterion': ['gini', 'entropy']
}

In [ ]:
print("--- Decision Tree: Baseline ---")
# Even for baseline, we use a pipeline with scaler for consistency, though not strictly needed for trees.
pipeline_tree = ImbPipeline([
    ('scaler', StandardScaler()),
    ('model', DecisionTreeClassifier(random_state=42))
])

model_baseline = model_select(pipeline_tree, param_grid_tree, x_train, y_train, x_test, y_test)

In [ ]:
print("\n--- Decision Tree: NearMiss ---")
# Scaler -> NearMiss -> Model
pipeline_tree_nm = ImbPipeline([
    ('scaler', StandardScaler()),
    ('sampler', NearMiss(version=1)),
    ('model', DecisionTreeClassifier(random_state=42))
])

tree_nm = model_select(pipeline_tree_nm, param_grid_tree, x_train, y_train, x_test, y_test)

In [ ]:
print("\n--- Decision Tree: SMOTE ---")
# Scaler -> SMOTE -> Model
pipeline_tree_smote = ImbPipeline([
    ('scaler', StandardScaler()),
    ('sampler', SMOTE(random_state=42)),
    ('model', DecisionTreeClassifier(random_state=42))
])

model_smote = model_select(pipeline_tree_smote, param_grid_tree, x_train, y_train, x_test, y_test)

### Decision Tree Analysis
- **Baseline**: F1 Score: 0.6020, ROC AUC: 0.8441
- **SMOTE**: F1 Score: 0.5949, ROC AUC: 0.8539
- **Observation**: The Decision Tree performs relatively well. SMOTE slightly improved the ROC AUC (0.844 -> 0.854) but slightly decreased the F1 score. This suggests that while SMOTE helps in separating the classes generally (AUC), it might be introducing some false positives that affect precision (lowering F1). NearMiss performed significantly worse, likely discarding too much valuable information.

## Random Forest

In [ ]:
param_grid_forest = {
    'model__n_estimators': [20, 50, 100, 200, 300, 400],
    'model__max_depth': [10, 20, 30, 40],
    'model__min_samples_split': [2, 5, 10]
}

In [ ]:
print("--- Random Forest: Baseline ---")
pipeline_forest = ImbPipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42))
])

best_rf_base = model_select(pipeline_forest, param_grid_forest, x_train, y_train, x_test, y_test)

In [ ]:
print("\n--- Random Forest: NearMiss ---")
pipeline_forest_nm = ImbPipeline([
    ('scaler', StandardScaler()),
    ('sampler', NearMiss(version=1)),
    ('model', RandomForestClassifier(random_state=42))
])

best_rf_nm = model_select(pipeline_forest_nm, param_grid_forest, x_train, y_train, x_test, y_test)

In [ ]:
print("\n--- Random Forest: SMOTE ---")
pipeline_forest_smote = ImbPipeline([
    ('scaler', StandardScaler()),
    ('sampler', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42))
])

best_rf_smote = model_select(pipeline_forest_smote, param_grid_forest, x_train, y_train, x_test, y_test)

### Random Forest Analysis
- **Baseline**: F1 Score: 0.5877, ROC AUC: 0.8649
- **SMOTE**: F1 Score: 0.6172, ROC AUC: 0.8626
- **Observation**: Random Forest with SMOTE is the top performer. It achieved the highest F1 score (0.6172) among all models while maintaining a very high ROC AUC (0.8626). The ensemble nature of Random Forest combined with SMOTE's synthetic data generation effectively handles the class imbalance, providing a robust model.

## Logistic Regression

In [ ]:
param_grid_lr = [
    {
        'model__penalty': ['l2'],
        'model__C': [0.01, 0.1, 1, 10, 100],
        'model__class_weight': ['balanced', None],
        'model__solver': ['lbfgs']
    },
    {
        'model__penalty': ['l1'],
        'model__C': [0.01, 0.1, 1, 10, 100],
        'model__class_weight': ['balanced', None],
        'model__solver': ['liblinear']
    }
]

In [ ]:
print("--- Logistic Regression: Baseline ---")
pipeline_lr = ImbPipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(random_state=42, max_iter=4000))
])

best_lr_base = model_select(pipeline_lr, param_grid_lr, x_train, y_train, x_test, y_test)

In [ ]:
print("\n--- Logistic Regression: NearMiss ---")
pipeline_lr_nm = ImbPipeline([
    ('scaler', StandardScaler()),
    ('sampler', NearMiss(version=1)),
    ('model', LogisticRegression(random_state=42, max_iter=4000))
])

best_lr_nm = model_select(pipeline_lr_nm, param_grid_lr, x_train, y_train, x_test, y_test)

In [ ]:
print("\n--- Logistic Regression: SMOTE ---")
pipeline_lr_smote = ImbPipeline([
    ('scaler', StandardScaler()),
    ('sampler', SMOTE(random_state=42)),
    ('model', LogisticRegression(random_state=42, max_iter=4000))
])

best_lr_smote = model_select(pipeline_lr_smote, param_grid_lr, x_train, y_train, x_test, y_test)

### Logistic Regression Analysis
- **Baseline**: F1 Score: 0.5042, ROC AUC: 0.7805
- **SMOTE**: F1 Score: 0.5100, ROC AUC: 0.7795
- **Observation**: Logistic Regression lags behind the tree-based models. Even with SMOTE and proper scaling (which we fixed in this notebook), the linear decision boundary is likely insufficient to capture the complex relationships in this dataset. The F1 score hovers around 0.51, which is significantly lower than the Random Forest's 0.61.

## 4. General Conclusion
Based on the comprehensive testing of Decision Trees, Random Forests, and Logistic Regression, using Baseline, NearMiss, and SMOTE strategies:

**The Best Model: Random Forest with SMOTE**
- **F1 Score**: 0.6172
- **ROC AUC**: 0.8626

**Why?**
1.  **Performance**: It achieves the best balance of Precision and Recall (F1 Score) and has excellent discriminatory power (ROC AUC).
2.  **Robustness**: Random Forests are less prone to overfitting than single Decision Trees.
3.  **Data Handling**: The combination of SMOTE (to address imbalance) and the pipeline approach (to ensure correct scaling and validation) proved most effective.

**Recommendation**:
We should proceed with the **Random Forest model trained with SMOTE**. It offers the most reliable predictions for identifying customers at risk of churning.